In [1]:
%load_ext autoreload
%autoreload 2

import zaber_pvt
import zaber_motion # May not need

In [4]:
zpvt=zaber_pvt.ZaberPVT()
zpvt.connect()


Found 3 devices


In [24]:
.get_position()

0.0

In [34]:
ax=zpvt.devices[0].get_axis(3)

In [35]:
import zaber

In [37]:
ax.get_position("mm")

62.999937499999966

In [12]:
# USe an odd number of pts seemed to work better to prevent PVT error
# (with weird velocities near center?)

# mult scales from 0-1 to mmmm.
# 50=12
# 75=17.5
zpvt.setup_zlut([0,30],[0,70],duration_sec=3,npts=31,mult=75)
#zpvt.move3()


In [13]:
zpvt.to_start3()

In [10]:
zpvt.sweep3()

In [38]:
zpvt.pvt_buffer.get_content()

ConnectionClosedException: ConnectionClosedException: Connection has been closed

In [171]:
zpvt.home3()

In [182]:
zpvt.live_pvt.point(*zpvt.start_pos)

In [181]:
zpvt.live_pvt.call(zpvt.pvt_buffer)

In [168]:
print( zpvt.start_pos[0])
#zpvt.devices[2].get_axis(1).move_absolute(0)

[{'_value': 0.0, '_unit': <Units.LENGTH_MILLIMETRES: 'Length:millimetres'>}, {'_value': 17.546666766076648, '_unit': <Units.LENGTH_MILLIMETRES: 'Length:millimetres'>}, {'_value': 0.0, '_unit': <Units.LENGTH_MILLIMETRES: 'Length:millimetres'>}]


In [78]:
pos3=[0,0,0]
p0=[zaber_motion.Measurement(p1,zaber_motion.Units.LENGTH_MILLIMETRES) for p1 in pos3]
v0=[zaber_motion.Measurement(p1,zaber_motion.Units.VELOCITY_MILLIMETRES_PER_SECOND) for p1 in pos3]

zpvt.live_pvt.point(p0, v0,
                    time=zaber_motion.Measurement(3, zaber_motion.Units.TIME_SECONDS))

In [33]:
zpvt.start_pos

[[{'_value': 0, '_unit': <Units.LENGTH_MILLIMETRES: 'Length:millimetres'>},
  {'_value': 21.056000119292, '_unit': <Units.LENGTH_MILLIMETRES: 'Length:millimetres'>},
  {'_value': 0, '_unit': <Units.LENGTH_MILLIMETRES: 'Length:millimetres'>}],
 [None, None, None],
 {'_value': 2.0, '_unit': <Units.TIME_SECONDS: 'Time:seconds'>}]

In [8]:

#/1 pvt 2 point abs p 0 39242 20157 v 0 0 0 t 1000

In [5]:
zpvt.pvt_buffer.get_content()

['setup store 1 3',
 'io set do 1 1',
 'point abs p 1186 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 2371 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 3557 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 4743 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 5929 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 7114 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 8300 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 9486 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 10672 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 11857 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 13043 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 14229 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 15415 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 16600 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 17786 0 0 v 0 0 0 t 58.8',
 'io set do 1 0',
 'point abs p 18972 0 0 v 0 0 0 t 58.8',
 'io set do 1 1',
 'point abs p 20157 0 0 v 

In [9]:
zpvt.live_pvt.point([0,39242,20157],[0,0,0], 2000)

TypeError: Provided value is not Measurement.

In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from zaber_motion import Units, Library, Measurement
from zaber_motion.ascii import Connection, AllAxes
from zaber_motion import *
import pandas as pd

In [3]:
Library.enable_device_db_store()
connection=Connection.open_serial_port("COM4")  # confirm that this is the right serial port

device_list = connection.detect_devices()
print("Found {} devices".format(len(device_list)))

device1 = device_list[0] 

Found 3 devices


In [4]:
device2 = device_list[2] # Mirror rotation

In [95]:
df_zlut=zaber_pvt.read_table()
zaber_pvt.table_to_pvt(zpvt.devices[0],df_zlut)

array([1, 2, 3])

In [102]:
zpvt.devices[0].all_axes.home()

In [112]:
zaber_pvt.make_table()

AttributeError: module 'zaber_pvt' has no attribute 'make_table'

In [5]:
# !python -m pip install -U zaber_motion

In [71]:
df_zlut=pd.read_excel('z_table.xlsx')
df_zlut.head()

NO_VELOCITY = True
# True=Let Zaber figure out what the velocities should be
# False=Use computed vels from xls. But the sign is important, unlike in xls.

step_size=50 # Can go through subsets of table with larger #s
idxs_subset=np.arange(0,50,step_size)
df_zlut=df_zlut.iloc[idxs_subset,:]

# Times are a period, not absolute time
# Assume they are equally spaced in time, and use the first one.
df_zlut.iloc[:,0]=np.diff( df_zlut.iloc[:,0] )[0]


# https://software.zaber.com/motion-library/docs/guides/pvt: 
# Make the velocity of the last one zero: necessary, it seems !

# Let Zaber figure out what the velocities should be
if NO_VELOCITY:
    df_zlut.iloc[:,1]=None
    df_zlut.iloc[0,1]=0
    df_zlut.iloc[-1,1]=0 #len(df_zlut.index),"Velocity (mm/s)"]=0

    #df_zlut.iloc[df_zlut["Displacement(mm)"]==0,1]=0
print( df_zlut)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
ax1_sweep_lims=[0,10] # mm
ax3_sweep_lims=[10,0] # mm

In [9]:
#ax0=device1.get_axis(2)

pvt = device1.get_pvt(1)
pvt.disable()

pvt_buffer = device1.get_pvt_buffer(1)
pvt_buffer.erase()

NDIMS=3
arr_axes=np.arange(NDIMS)+1
# set up PVT to store points to PVT buffer 1 and
# to use the first axis for unit conversion
pvt.setup_store(pvt_buffer, *arr_axes )

# add PVT points from LUT in Excel file
for index, row in df_zlut.iterrows():

    # Add each point in a loop
    if NDIMS==1:
        poses=[Measurement(row["Displacement(mm)"], Units.LENGTH_MILLIMETRES)]
    elif NDIMS==2:
        ax1_pos = ax1_sweep_lims[0] + (index)*(ax1_sweep_lims[1]-ax1_sweep_lims[0])/df_zlut.index[-1]
        poses=[Measurement(ax1_pos, Units.LENGTH_MILLIMETRES),
               Measurement(row["Displacement(mm)"], Units.LENGTH_MILLIMETRES)]
    else: # Assume 3
        ax1_pos = ax1_sweep_lims[0] + (index)*(ax1_sweep_lims[1]-ax1_sweep_lims[0])/df_zlut.index[-1]
        ax3_pos = ax3_sweep_lims[0] + (index)*(ax3_sweep_lims[1]-ax3_sweep_lims[0])/df_zlut.index[-1]
        poses=[Measurement(ax1_pos, Units.LENGTH_MILLIMETRES),
               Measurement(row["Displacement(mm)"]/10.0, Units.LENGTH_MILLIMETRES),
               Measurement(ax3_pos, Units.LENGTH_MILLIMETRES) ];
        
    if row["Velocity (mm/s)"] is None:
        vels=[None]*NDIMS
    else:
        vels=[Measurement(row["Velocity (mm/s)"], Units.VELOCITY_MILLIMETRES_PER_SECOND)]*NDIMS
           
    # Toggle the DIO (first channel): at 0 will be 1, at 1 will be 0, at 2 will be 1, etc..
    # use +1 mod 2. TODO (this is pretty hacky and inflexible)
    pvt.set_digital_output(1,(index+1)%2)
    pvt.point(poses, vels , Measurement(row["Time(s)"], Units.TIME_SECONDS) )
    #time.sleep(0.01)
pvt.set_digital_output(1,0)

# finish writing to the buffer
pvt.disable()

print(pvt_buffer)
print( pvt )
print( pvt_buffer.get_content() )

Device 1 -> PVT Sequence 1 (Disabled)
['setup store 1 3', 'io set do 1 1', 'point abs p 0 3924 20157 v 0 0 0 t 600.0', 'io set do 1 1', 'point abs p 5039 1369 15118 v 13761 -5203 -13761 t 600.0', 'io set do 1 1', 'point abs p 10079 113 10079 v 13761 -1522 -13761 t 600.0', 'io set do 1 1', 'point abs p 15118 254 5039 v 13761 2277 -13761 t 600.0', 'io set do 1 1', 'point abs p 20157 1781 0 v 0 0 0 t 600.0', 'io set do 1 0', 'setup disable']


In [10]:
# execute:
pvt = device1.get_pvt(2)
pvt.disable()
pvt.setup_live(*arr_axes) # Execute on axis Z

# the sequence will now execute the stored points
pvt.call(pvt_buffer)

#print (pvt.

In [65]:
pvt.call(pvt_buffer)
amt_deg=45
device2.get_axis(1).move_absolute(amt_deg, Units.ANGLE_DEGREES,
                                  velocity=amt_deg/3.0, velocity_unit=Units.ANGULAR_VELOCITY_DEGREES_PER_SECOND,
                                  wait_until_idle=False)

In [ ]:
pvt.call(pvt_buffer)

In [64]:
device2.get_axis(1).home()

In [54]:
help( device2.get_axis(1).move_absolute )

Help on method move_absolute in module zaber_motion.ascii.axis:

move_absolute(position: float, unit: Union[zaber_motion.units.Units, Literal['m', 'cm', 'mm', 'µm', 'um', 'nm', 'in', '°', 'deg', 'rad']] = <Units.NATIVE: ''>, wait_until_idle: bool = True, velocity: float = 0, velocity_unit: Union[zaber_motion.units.Units, Literal['m/s', 'cm/s', 'mm/s', 'µm/s', 'um/s', 'nm/s', 'in/s', '°/s', 'deg/s', 'rad/s']] = <Units.NATIVE: ''>, acceleration: float = 0, acceleration_unit: Union[zaber_motion.units.Units, Literal['m/s²', 'm/s^2', 'cm/s²', 'cm/s^2', 'mm/s²', 'mm/s^2', 'µm/s²', 'um/s^2', 'nm/s²', 'nm/s^2', 'in/s²', 'in/s^2', '°/s²', 'deg/s^2', 'rad/s²', 'rad/s^2']] = <Units.NATIVE: ''>) -> None method of zaber_motion.ascii.axis.Axis instance
    Move axis to absolute position.
    
    Args:
        position: Absolute position.
        unit: Units of position.
        wait_until_idle: Determines whether function should return after the movement is finished or just started.
        velocit

In [48]:
import time
t1=time.time()
amt_deg=90
device2.get_axis(1).move_absolute(amt_deg, Units.ANGLE_DEGREES, velocity=amt_deg/3.0, wait_until_idle=False)
pvt.call(pvt_buffer)
print ( time.time() - t1)

0.015173673629760742


In [40]:
print ( time.time() - t1)

9.398441314697266


In [27]:
time.time()-t1

1.8900182247161865

In [32]:
d3=device_list[2]

amt_deg=10
pvt.call(pvt_buffer)
device2.get_axis(1).move_absolute(amt_deg, Units.NATIVE, velocity=amt_deg/3.0, wait_until_idle=False)
#d3.get_axis(1).move_absolute(amt_deg, Units.ANGLE_DEGREES, velocity=amt_deg/3.0, wait_until_idle=False)
#,wait_until_idle=False)

In [13]:
[device1.get_axis(n).home() for n in [1,2,3]]

[None, None, None]

In [14]:
axs=[device1.get_axis(n) for n in [1,2,3]]

In [17]:
axs[0].move_absolute(0)
axs[1].move_absolute(40000)
axs[2].move_absolute(20000)

In [19]:
ax1.home()
ax2.home()
ax3.home()

NameError: name 'ax1' is not defined

In [246]:
[ax1.get_position() for ax1 in axs]

[20157.0, 17809.0, 10.0]

In [4]:
ax1=device1.get_axis(1)
ax1.get_state()

'{"context":{"v":3,"type":"peripheral","fw":{"major":7,"minor":33,"build":14091}},"axis":{"peripheralId":"70131","serial":"112556","settings":{"cloop.continuous.enable":"0","cloop.displace.tolerance":"128","cloop.enable":"1","cloop.recovery.enable":"0","cloop.stall.action":"0","cloop.stall.detect.mode":"0","cloop.stall.tolerance":"512","cloop.timeout":"100","driver.current.hold":"35","driver.current.run":"70","ictrl.advance.a":"0.000000000","ictrl.advance.offset":"4096.000000000","ictrl.afcff.inductance":"0.088000000","ictrl.afcff.ke":"7.781229568","ictrl.afcff.ki":"0.000000000","ictrl.afcff.max":"48000.000851968","ictrl.afcff.ss":"0.000000000","ictrl.afcff.ss.max":"0.999899840","ictrl.ff.kd":"0.088000000","ictrl.ff.kp":"0.000999999","ictrl.gain.coldmult":"0.299999968","ictrl.pi.ki":"0.000028998","ictrl.pi.kp":"0.011596000","ictrl.type":"6","knob.dir":"0","knob.distance":"640","knob.enable":"1","knob.maxspeed":"153600","knob.mode":"0","knob.speedprofile":"2","limit.approach.maxspeed":"

In [7]:
ax3=device1.get_axis(3)
print( ax3.get_state() )


{"context":{"v":3,"type":"peripheral","fw":{"major":7,"minor":33,"build":14091}},"axis":{"peripheralId":"70134","serial":"112555","settings":{"cloop.continuous.enable":"0","cloop.displace.tolerance":"128","cloop.enable":"1","cloop.recovery.enable":"0","cloop.stall.action":"0","cloop.stall.detect.mode":"0","cloop.stall.tolerance":"512","cloop.timeout":"100","driver.current.hold":"35","driver.current.run":"70","ictrl.advance.a":"0.000000000","ictrl.advance.offset":"4096.000000000","ictrl.afcff.inductance":"0.088000000","ictrl.afcff.ke":"7.781229568","ictrl.afcff.ki":"0.000000000","ictrl.afcff.max":"48000.000851968","ictrl.afcff.ss":"0.000000000","ictrl.afcff.ss.max":"0.999899840","ictrl.ff.kd":"0.088000000","ictrl.ff.kp":"0.000999999","ictrl.gain.coldmult":"0.299999968","ictrl.pi.ki":"0.000028998","ictrl.pi.kp":"0.011596000","ictrl.type":"6","knob.dir":"0","knob.distance":"640","knob.enable":"1","knob.maxspeed":"153600","knob.mode":"0","knob.speedprofile":"2","limit.approach.maxspeed":"1

In [178]:
ax3.home()

In [180]:
ax3.move_absolute(10)

In [18]:
device1.get_axis(3)

Axis 3 (LSQ150B-E01T3A) -> Device 1 SN: 112052 (X-MCC3 (rev 2/3)) -> Connection 1 (ASCII Serial port: COM4)

In [66]:
ax_r1=zpvt.devices[1].get_axis(1)

In [67]:
ax_r1.move_relative(-2000)